In [2]:
import pandas as pd

In [3]:
vamps = pd.read_csv("vampires.csv")
vamps.head()

,Unnamed: 0,Clan,Generation,Sire,Name,Link,Childer
0,0,Ventrue,9th,T.J. McMillan,Leon White,https://whitewolf.fandom.com/wiki/Leon_White,NaN
1,1,Followers of Set,5th (Mechanically function as 8th),Nefertiti,Aabbt Kindred,https://whitewolf.fandom.com/wiki/Aabbt_Kindred,NaN
2,2,Malkavian,NaN,NaN,Aaron,https://whitewolf.fandom.com/wiki/Aaron,NaN
3,3,NaN,NaN,NaN,Aaron MacAndrews,https://whitewolf.fandom.com/wiki/Aaron_MacAnd...,Hilda MacAndrews
4,4,Toreador,10th,Regina Blake,Rose Abawi,https://whitewolf.fandom.com/wiki/Rose_Abawi,NaN


In [4]:
vamps.drop(vamps.columns[0], axis = 'columns', inplace= True) #drop an extra index column
vamps.head()

,Clan,Generation,Sire,Name,Link,Childer
0,Ventrue,9th,T.J. McMillan,Leon White,https://whitewolf.fandom.com/wiki/Leon_White,NaN
1,Followers of Set,5th (Mechanically function as 8th),Nefertiti,Aabbt Kindred,https://whitewolf.fandom.com/wiki/Aabbt_Kindred,NaN
2,Malkavian,NaN,NaN,Aaron,https://whitewolf.fandom.com/wiki/Aaron,NaN
3,NaN,NaN,NaN,Aaron MacAndrews,https://whitewolf.fandom.com/wiki/Aaron_MacAnd...,Hilda MacAndrews
4,Toreador,10th,Regina Blake,Rose Abawi,https://whitewolf.fandom.com/wiki/Rose_Abawi,NaN


### Clans

In [5]:
#vamps["Clan"] = vamps["Clan"].apply(func=lambda x: str(x).removesuffix(" antitribu")) #remove all antitribu and move them to the main clan

vamps["Clan"] = vamps["Clan"].str.lower()

main_clans = ["banu haqim","brujah","gangrel",
              "hecata","lasombra","malkavian",
              "the ministry","nosferatu","ravnos",
              "salubri","toreador","tremere",
              "tzimisce","ventrue", "caitiff", "baali"]

def find_main_clan(s):
    for clan in main_clans:
        if clan in s:
            return clan 
    return s

vamps["Clan"] = vamps["Clan"].apply(lambda x: find_main_clan(str(x)))


def handle_edge_cases(s):

    if "thin" in s:
        return "thinblood" 
    if ("ministry" in s) or ("set" in s) or ("serpents" in s):
        return "followers of set"
    if ("assimite" in s) or ("assamite" in s):
        return "banu haqim"
    if ("giovanni" in s) or ("cappadocian" in s) or ("nagaraja" in s) or ("samedi" in s):
        return "hecata"
    if ("nan" in s) or ("[]" in s):
        return "unknown"
    if ("cacophony" in s):
        return "toreador"
    if ("gargoyle" in s):
        return "tzimisce"
    if ("kiasyd" in s):
        return "lasombra"

    return s 

vamps["Clan"] = vamps["Clan"].apply(lambda x: handle_edge_cases(str(x)))

clan_population = vamps["Clan"].value_counts()

low_population = list(clan_population[clan_population < 5].index)

#print(low_population)
vamps = vamps[~vamps["Clan"].isin(low_population)]


In [6]:
vamps["Clan"].value_counts()
vamps["Clan"].value_counts().to_csv("unique_clans.csv")

### Generation

In [7]:
import re
import numpy as np

def select_generation(gen_str):

    gen_str = str(gen_str)

    gen_str = gen_str.replace("V5", "") #deal with the edge cases where V5 system is mentioned
    gen_str = re.sub(r'\[\d+\]', "", gen_str) #deal with source reference (of the type [4])
    gen = re.findall(r'\d+', gen_str)

    gen = [int(x) for x in gen if int(x) <= 16 or int(x) > 0]

    if not gen: 
        return np.nan

    if "diab" in gen_str: #check for diablerie 
        gen = min(gen)
    else: 
        gen = max(gen)

    return gen

vamps["Generation"] = vamps["Generation"].apply(lambda x: select_generation(x))
#generational_digits.to_csv("generational_digits2.csv")

In [8]:
#vamps[~vamps["Generation"].isna()]
vamps["Generation"]

0        9.0
1        8.0
2        NaN
3        NaN
4       10.0
        ... 
2112     NaN
2113     2.0
2114     NaN
2115     NaN
2116     NaN
Name: Generation, Length: 2102, dtype: float64

### Individual Clans

In [9]:
brujah = vamps[vamps["Clan"] == "brujah"]

In [10]:
brujah[~brujah["Sire"].isna()]

,Clan,Generation,Sire,Name,Link,Childer
43,brujah,8.0,Edward Scott,Akawa,https://whitewolf.fandom.com/wiki/Akawa,Wrecker
60,brujah,6.0,Agonistes,Al-Muntaqim,https://whitewolf.fandom.com/wiki/Al-Muntaqim,NaN
101,brujah,7.0,Sigfried,Andrei (Brujah),https://whitewolf.fandom.com/wiki/Andrei_(Brujah),NaN
106,brujah,9.0,Christof Romuald,Anezka,https://whitewolf.fandom.com/wiki/Anezka,NaN
112,brujah,4.0,Brujah Antediluvian,Anis,https://whitewolf.fandom.com/wiki/Anis,NaN
...,...,...,...,...,...,...
2045,brujah,11.0,Jake Almerson,Tom Weaver,https://whitewolf.fandom.com/wiki/Tom_Weaver,NaN
2064,brujah,7.0,"""Unknown""",Aaron Wilkshire,https://whitewolf.fandom.com/wiki/Aaron_Wilkshire,NaN
2076,brujah,9.0,Akawa,Wrecker,https://whitewolf.fandom.com/wiki/Wrecker,NaN
2087,brujah,6.0,Clear Bear,Yaryan,https://whitewolf.fandom.com/wiki/Yaryan,NaN


### Graphs

In [11]:
red_gradient = {
    1: "#8b0000",    # Darkest Red
    2: "#920c0c",
    3: "#991919",
    4: "#a02626",
    5: "#a83333",
    6: "#af3f3f",
    7: "#b64c4c",
    8: "#bd5959",
    9: "#c46666",
    10: "#cc7272",
    11: "#d37f7f",
    12: "#da8c8c",
    13: "#e29999",
    14: "#e9a5a5",
    15: "#f0b2b2",
    16: "#f7bfbf",
    17: "#ffcccc"     # Lightest Red
}

In [12]:
def generate_edges_sires(clan):
    edges = []
    for row in range(clan.shape[0]):
        
        #print(clan.iloc[row]["Sire"])
        #print(clan.iloc[row]["Sire"])
        #print("Akawa" in list(clan["Name"]))
        if clan.iloc[row]["Sire"] in list(clan["Name"]):
            
            edges.append((clan.iloc[row]["Sire"], clan.iloc[row]["Name"]))

    return edges

#print(brujah.head())
brujah_edges = generate_edges_sires(brujah)

In [237]:
from pyvis.network import Network
from pyvis.options import Layout
import networkx as nx
from matplotlib import pyplot as plt

#G = nx.complete_multipartite_graph()
#pos = nx.multipartite_layout(G)

#layout = Layout()
#layout.set_tree_spacing(10)

#net = Network(directed=True , layout="multipartite_layout", bgcolor="#222222")
net = Network(height="800px", directed=True, bgcolor="#222222", layout=True)
#net.show_buttons(["physics"])

brujah_temp = brujah.sort_values(by="Generation")
brujah_temp

for index, vamp in brujah_temp.iterrows():
    #print(vamp["Name"])
    if not np.isnan(vamp["Generation"]): 
        level = vamp["Generation"]
        net.add_node(n_id=vamp["Name"],
                     label=vamp["Name"] + "\n" + str(int(level)) + " gen",
                     level = level, 
                     color=red_gradient[level],
                     shape = "box",
                     title = vamp["Link"], #title=f"<a href='{vamp["Link"]}' target='_blank'>{vamp["Name"]} \n {int(level)} gen</a>",
                     font = {"color":"white"})
    


for index, vamp in brujah_temp.iterrows():
    if np.isnan(vamp["Generation"]):
        level = np.random.binomial(n = 15, p = 0.5)
        net.add_node(n_id=vamp["Name"],
                     level = level, 
                     color="#BEBEBE")

for edge in brujah_edges:
    net.add_edge(edge[0], edge[1])


#net.add_edges(brujah_edges)
#net.from_nx(G)


In [238]:
updown = False
if updown: 
    net.set_options("""options= {
                        "layout": {
                            "hierarchical": {
                            "enabled": true,
                            "sortMethod": "directed",
                            "levelSeparation": 80,
                            "nodeSpacing": 20,
                            "treeSpacing": 50,
                            "direction": "LR"
                            }
                        }
                        }
                    """)
else:
    net.set_options("""options= {
            "layout": {
                "hierarchical": {
                "enabled": true,
                "nodeSpacing": 20,
                "treeSpacing": 50,
                "direction": "LR"
                }
            }
        }
            """)
net.show("mygraph.html")

Error: Failed to open Wayland display, fallback to X11. WAYLAND_DISPLAY='wayland-0' DISPLAY=':0'


In [ ]:
#net.show_buttons(["layout", "physics"])
#net.show("mygraph.html")

Error: Failed to open Wayland display, fallback to X11. WAYLAND_DISPLAY='wayland-0' DISPLAY=':0'


In [ ]:

with open("mygraph.html", 'r', encoding='utf-8') as f:

    source_code = f.read()

    idx = source_code.index('<html>\n ') + len('<html>\n ')

    source_code_updated = source_code[:idx] + '<body style="background-color:#222222;">\n ' + source_code[idx:]

    
    idx = source_code_updated.index('<style type="text/css">') + len('<style type="text/css">')

    source_code_updated = source_code_updated[:idx] + "\n\t\t\t#mycard {\n \t\t\t\tbackground-color: #222222;\n\t\t\t\n  border: 1px solid #222222;\n}\n #loadingBar { display: none !important;}\n" + source_code_updated[idx:]

    idx = source_code_updated.index('<div class="card" style="width: 100%">') + len('<div ')

    source_code_updated = source_code_updated[:idx] + 'id="mycard" ' + source_code_updated[idx:]


    idx = source_code_updated.index("border: 1px solid lightgray;") + len('border: 1px solid ')
    source_code_updated = source_code_updated[:idx] + "#222222;" + source_code_updated[idx+len("#222222;")+2:]

    idx = source_code_updated.index("network = new vis.Network(container, data, options);") + len("network = new vis.Network(container, data, options);")
    func = """\n network.on("doubleClick", function(params) {
                        if (params.nodes.length > 0) {
                            var nodeId = params.nodes[0];
                            var node = nodes.get(nodeId);
                            console.log(node)
                            if (node && node.title) {
                                window.open(node.title, "_blank");
                            }
                        }
                    });\n""" 

    source_code_updated = source_code_updated[:idx] + func + source_code_updated[idx+len("network = new vis.Network(container, data, options);")+2:] #adds the double-click to the node


    idx = source_code_updated.index('network.once("stabilizationIterationsDone", function() {') + len('network.once("stabilizationIterationsDone", function() {') 
    
    func = """\n
    network.moveTo({
        position: {x: -400, y: -5700},
        scale: 0.8
    });"""

    source_code_updated = source_code_updated[:idx] + func + source_code_updated[idx:] #fixes the zoom

with open('mygraph.html', 'w', encoding='utf-8') as f:
    f.write(source_code_updated)
    source_code_updated




### Different Layout

In [ ]:
brujah_temp = brujah.sort_values(by="Generation")
brujah_temp

,Clan,Generation,Sire,Name,Link,Childer
1918,brujah,3.0,Brujah Antediluvian,Troile,https://whitewolf.fandom.com/wiki/Troile,"['Etheyra', 'Losario', 'Gersakkun of Tyre', 'M..."
112,brujah,4.0,Brujah Antediluvian,Anis,https://whitewolf.fandom.com/wiki/Anis,NaN
657,brujah,4.0,Troile,Etheyra,https://whitewolf.fandom.com/wiki/Etheyra,NaN
209,brujah,4.0,Brujah,Belit-Sheri,https://whitewolf.fandom.com/wiki/Belit-Sheri,Shalmath
865,brujah,4.0,Troile,Hannibal (Brujah),https://whitewolf.fandom.com/wiki/Hannibal_(Br...,Guillaume (515 CE)
...,...,...,...,...,...,...
1867,brujah,NaN,NaN,Thelonious,https://whitewolf.fandom.com/wiki/Thelonious,Thomas (ghoul)
1896,brujah,NaN,Unknown,Torque,https://whitewolf.fandom.com/wiki/Torque,NaN
1945,brujah,NaN,NaN,Torrance Urich,https://whitewolf.fandom.com/wiki/Torrance_Urich,NaN
1981,brujah,NaN,Philip Freeman,Velvet (VTM),https://whitewolf.fandom.com/wiki/Velvet_(VTM),NaN
